In [125]:
# This script has a faster way of collecting the data by simply looking up the firm names on the indexes but because
# it doesn't visit the company webpages on the LSE website it may be less reliable than the former script
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


df = pd.read_excel("input.xlsx")
cols = df["Company Name"].tolist()
cols = [i.upper() for i in cols]
for i in cols:
    print(i)


i = 1
list_of_urls = []
firm_names = []
penultimate_page = False


indexes = {"FTSE100":"ftse-100",
           "FTSE250" : "ftse-250",
          "AIM": "ftse-aim-all-share"}

FTSE100_firm_names = []
FTSE250_firm_names = []
AIM_firm_names = []

for index in indexes:

    print(f"\nNow collecting {index}")
    
    while True:
        response = requests.get(f"https://www.londonstockexchange.com/indices/{indexes[index]}/constituents/table?page={i}")
        soup = BeautifulSoup(response.text, 'html.parser')
        constituents = soup.find_all("a", class_="page-number")
        firms = soup.find_all("a", class_="dash-link black-link ellipsed")
        for firm in firms:
            firm_names.append(firm.get_text())
        print(response.url)
        list_of_urls.append(response.url)
        i += 1
        if index == "FTSE100":
            if i > 6:
                break
            continue        
        if penultimate_page == True:
            break
        if constituents[5].get_text() != "...":
            penultimate_page = True
            continue

    for firm_name in firm_names:
        firm_name = firm_name.split('ORD')
        firm_name = firm_name[0]
        firm_name = firm_name.strip()
        
        if index == "FTSE100":
            FTSE100_firm_names.append(firm_name)
        elif index == "FTSE250":
            FTSE250_firm_names.append(firm_name)
        elif index == "AIM":
            AIM_firm_names.append(firm_name)
        i = 1
        penultimate_page = False


print("\nNow collecting Small Cap")
        
sc_firm_names = []      

# works out how many pages there are to look for for small caps
response = requests.get("https://www.hl.co.uk/shares/stock-market-summary/ftse-small-cap?page=1")
soup = BeautifulSoup(response.text, 'html.parser')
constituents = soup.find_all("td", class_="align-center border-right-none")
page_no = constituents[0].get_text().strip()
page_no = page_no[::-1]
page_no = int(page_no[0])

for page in range(1,page_no+1):
    response = requests.get(f"https://www.hl.co.uk/shares/stock-market-summary/ftse-small-cap?page={page}")
    print(response.url)
    soup = BeautifulSoup(response.text, 'html.parser')
    constituents = soup.find_all("a", {'onclick':"if(typeof(scto)!='undefined'){scto.adv_trackevent('content','internal','SMX_page|click_ex-page|View factsheet main')};"})
    for i in constituents:
        sc_firm_names.append(i.get_text().upper())

output = []
counter = 0

for col in cols:
    counter += 1
    print(col)
    if col in FTSE100_firm_names:
        output.append("FTSE 100")
        continue
    if col in FTSE250_firm_names:
        output.append("FTSE 250")
        continue
    if col in AIM_firm_names:
        output.append("AIM")
        continue
    if col in sc_firm_names:
        output.append("Small Cap")
        continue
    output.append("Not found")

for i in output:
    print(i)
    
df = pd.DataFrame(output)
df.to_excel("output_file.xlsx")          

HUNTING PLC
PLEXUS HOLDINGS PLC
ENQUEST PLC
TOPPS TILES PLC
UP GLOBAL SOURCING HOLDINGS PLC
LEEDS GROUP PLC
MORGAN SINDALL
SPRINGFIELD PROPERTIES PLC
TOWN CENTRE SECURITIES PLC

Now collecting FTSE100
https://www.londonstockexchange.com/indices/ftse-100/constituents/table?page=1
https://www.londonstockexchange.com/indices/ftse-100/constituents/table?page=2
https://www.londonstockexchange.com/indices/ftse-100/constituents/table?page=3
https://www.londonstockexchange.com/indices/ftse-100/constituents/table?page=4
https://www.londonstockexchange.com/indices/ftse-100/constituents/table?page=5
https://www.londonstockexchange.com/indices/ftse-100/constituents/table?page=6

Now collecting FTSE250
https://www.londonstockexchange.com/indices/ftse-250/constituents/table?page=1
https://www.londonstockexchange.com/indices/ftse-250/constituents/table?page=2
https://www.londonstockexchange.com/indices/ftse-250/constituents/table?page=3
https://www.londonstockexchange.com/indices/ftse-250/constituent